In [2]:
import matplotlib.pyplot as plt 
import numpy as np
%matplotlib notebook

import matplotlib.units as munits
import matplotlib.dates as mdates
converter = mdates.ConciseDateConverter()
munits.registry[np.datetime64] = converter


In [3]:
my_data = np.genfromtxt('COVID19.csv', delimiter=',', converters={0: lambda x: np.datetime64(x)})

date = np.array([d[0] for d in my_data[1:]])
cases = np.array([d[1] for d in my_data[1:]])
deaths = np.array([d[2] for d in my_data[1:]])
casesVI = np.array([d[3] for d in my_data[1:]])

tests = np.array([d[4] for d in my_data[1:]])
yday = (date-np.datetime64('2020-01-01')).astype('float')


In [6]:
def fitgrowth(t, cases, ntofit=None):
    good = np.isfinite(t+cases)
    t = t[good]
    cases=cases[good]
    
    if ntofit is None:
        ntofit = y
    tofit = np.where(t>t[-1]-ntofit)[0]
    print(tofit)
    t0 = t[tofit[0]]
    p = np.polyfit(t[tofit]-t0, np.log(cases[tofit]), 1)
    tau = 1/p[0]
    cases0 = np.exp(p[1])
    doubletime = np.log(2) * tau
    return tau, cases0, doubletime, t0

<IPython.core.display.Javascript object>


[ 9 10 11 12 13 14]
[0 1 2 3]
[ 9 10 11 12 13 14]
[0 1 2 3]


/Users/jklymak/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: UserWarning: constrained_layout not applied.  At least one axes collapsed to zero width or height.
/Users/jklymak/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: UserWarning: constrained_layout not applied.  At least one axes collapsed to zero width or height.


In [8]:
fig, axs = plt.subplots(1, 2, figsize=(7, 4), constrained_layout=True)

for nn in range(2):
    ax = axs[nn]
    
    ax.semilogy(date, deaths, '.', label='Fatalities')

    good = np.where(np.isfinite(tests))[0]
    ax.semilogy(date[good], tests[good], 'd-')
    ax.text(date[good[0]], tests[good[0]], 'Provincial\nTests\n', fontsize='small', fontstyle='italic', ha='center')

    ax.semilogy(date, cases, '.', label='BC Cases')
    ax.set_ylim(1, 400)

    tau, cases0, doubletime, t0 = fitgrowth(yday, cases, ntofit=7)
    
    dd = np.arange('2020-03', '2020-05', dtype='datetime64[D]')
    ddy = (dd-np.datetime64('2020-01-01')).astype('float')
    possiblecases = cases0*np.exp((ddy-t0)/tau)
    possiblecases = np.clip(possiblecases, 0, 5e6)
    ax.plot(dd, possiblecases, zorder=0)
    ind = np.where(possiblecases>1e6)[0][0]
    ax.text(dd[ind], possiblecases[ind], f'  BC Doubling time: \n{np.log(2) * tau:1.2f}d ', 
            ha='right', fontsize='small', fontstyle='italic')

    # VI
    ax.semilogy(date, casesVI, '.', label='Vancouver Is.')

    tauVI, cases0VI, doubletimeVI, t0 = fitgrowth(yday, casesVI, ntofit=7)
    fitcases = cases0VI*np.exp((ddy-t0)/tauVI)
    fitcases = np.clip(fitcases, 0, 870297)
    ax.plot(dd, fitcases, zorder=0)

    ind = np.where(fitcases>1e3)[0][0]
    ax.text(dd[ind], fitcases[ind], f'      Vanc. Is.\n    Doubling time: \n   {np.log(2) * tauVI:1.2f}d ', 
            ha='left', fontsize='small', fontstyle='italic')


    if nn == 0:
        ax.set_ylim(1, 1000)
        ax.set_yscale('linear')    
        ax.legend(loc=2, fontsize='small')
    else:
        ax.set_yscale('log')
        ax.set_ylim(1, 1e7)
axs[0].set_ylabel('Cases')
axs[0].set_title(f'Date: {date[-1]}', loc='left')
axs[0].set_xlim([date[0], date[-1]+np.timedelta64(15, 'D')] )

fig.savefig('Cases.pdf')
fig.savefig('Cases.png')



$ c = a\ e^{\frac{t-t_0}{\tau}}$ 

$log(c) = log(a) + (t-t_0)/\tau$

$exp((t-t0)/tau) = 2$
$t-t0 = np.log(2) * tau$
